In [1]:
import numpy as np
import os
import re
import json
from scipy.sparse import coo_matrix,csr_matrix,csc_matrix,save_npz,load_npz
import string
from collections import Counter
import sqlite3

#### This notebook contains all the commands to push data into database
#### Indeed, all the data produced in the other notebooks is saved into json files
#### The aim of this notebook is to process each one of these json files and add the data into database

In [8]:
# this file contains all the source and target of each link in the articles
f = open('../Database/link_dict_inverse.json')
link_dict_inverse = json.load(f)
f.close()
link_dict_inverse={int(key):list(map(int, value)) for key,value in link_dict_inverse.items()}

conn = sqlite3.connect('../Database/Database.db')
cursor = conn.cursor()
# cursor.execute("Drop Table Articles")
create_db="CREATE TABLE Link_Dict_Inverse(ID_target NUMERIC,ID_source NUMERIC);"
cursor.execute(create_db)
index="CREATE INDEX index_id_target ON Link_Dict_Inverse (ID_target);"
cursor.execute(index)

for key,value in link_dict_inverse.items():
    for source in value:
        request='Insert into Link_Dict_Inverse Values (%d,%d)'%(key,source)
        cursor.execute(request)
conn.commit()
conn.close()

In [7]:
# this file contains all the informations of each article
# ID ROW = The row in the tf-idf, M, C matrices ; ID article : the real id of the article ; 
# title = title of the article ; url :url; location : absolute path towards the file containing the wikipedia article;
# line_index : the line into the file (each line of the file is a wikipedia article) 
f = open('../Database/dataset.json')
dataset = json.load(f)
f.close()
dataset={int(key):[value[0],int(value[1]),value[2],value[3],value[4]] for key,value in dataset.items()}

conn = sqlite3.connect('../Database/Database.db')
cursor = conn.cursor()
# cursor.execute("Drop Table Articles")
create_db="CREATE TABLE Articles(ID_ROW NUMERIC,ID_ARTICLE NUMERIC,Title TEXT,URL TEXT,Location TEXT,Line_index NUMERIC);"
cursor.execute(create_db)
index="CREATE INDEX index_id_article2 ON Articles (ID_ARTICLE);"
cursor.execute(index)
index="CREATE INDEX index_id ON Articles (ID_ROW);"
cursor.execute(index)


sq="'" #single quote
for key,value in dataset.items():
    tmp=value[1].replace("'"," ")
    request='Insert into Articles Values (%s,%s,%s,%s,%s,%s)'%(key,value[0],sq+tmp+sq,sq+value[2]+sq,sq+value[3]+sq,value[4])
    cursor.execute(request)
conn.commit()
conn.close()

('Hyacinthe Cartuyvels (1849-1897)', 'https://fr.wikipedia.org/wiki?curid=6797291')


In [4]:
# This file contains the page rank vector of the articles
r=np.load('../Database/r.npy')

conn=sqlite3.connect('../Database/Database.db')
cursor=conn.cursor()
cursor.execute("DROP TABLE R")
create_db="CREATE TABLE R(ID NUMERIC,Value TEXT);"
cursor.execute(create_db)
index="CREATE INDEX index_id_r ON R(ID);"
cursor.execute(index)

sq="'"
for i in range(len(r)):
    request='Insert into R Values (%d,%s)'%(i,sq+str(r[i,0])+sq)
    cursor.execute(request)
conn.commit()
conn.close()

In [6]:
# this file contains for each article its id and the index of the row associated in all matrices (R for Page Rank,
# C for tf-idf, M for tf-idf, word2vec vectors)
f = open('../Database/indices_dict.json')
indices_dict = json.load(f)
f.close()

conn=sqlite3.connect('../Database/Database.db')
cursor=conn.cursor()
cursor.execute("Drop table Transitional_id_article_index_R")
create_db="CREATE TABLE Transitional_id_article_index_R(ID_ARTICLE NUMERIC,INDEX_ROW NUMERIC);"
cursor.execute(create_db)
index="CREATE INDEX index_id_article ON Transitional_id_article_index_R(ID_ARTICLE);"
cursor.execute(index)

for key,value in indices_dict.items():
    request='Insert into Transitional_id_article_index_R Values (%d,%d)'%(int(key),value)
    cursor.execute(request)
conn.commit()
conn.close()

In [3]:
# new_voc_dict contains all the words of the tf-idf matrix (each word is a column) as key 
# and the index of the column as value
# M is the M matrix in the tf-idf process. Contains for each cell the number of times a word is seen in an article

f = open('../Database/new_voc_dict.json')
new_voc_dict = json.load(f)
f.close()
M = load_npz("../Database/M.npz")
somme=np.sum(M,axis=0)

conn=sqlite3.connect('../Database/Database.db')
cursor=conn.cursor()
cursor.execute("Drop table Vocabulary")
create_db="CREATE TABLE Vocabulary(WORD TEXT,INDEX_COLUMN NUMERIC, M_SUM NUMERIC);"
cursor.execute(create_db)
index="CREATE INDEX index_id_word ON Vocabulary(WORD);"
cursor.execute(index)

sq="'"
count=0

for key,value in new_voc_dict.items():
    request='Insert into Vocabulary Values (%s,%d,%d)'%(sq+key+sq,value,somme[0,value])
    cursor.execute(request)
    count+=1
    if count%100000==0:
        print(count)
conn.commit()
conn.close()

100000
200000
300000
